In [1]:
import skopt
from skopt import gp_minimize, forest_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence
from skopt.plots import plot_objective, plot_evaluations
from skopt.utils import use_named_args
from torch.optim.lr_scheduler import ReduceLROnPlateau

from data_functions import build_loader
from model_functions import build_segmentation_model
from training_functions import train

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
from training_functions import *
from tensorboardX import SummaryWriter
import numpy as np
from imgaug import augmenters as iaa
import imgaug as ia


In [1]:
!pip install scikit-optimize
!pip install imgaug
!pip install opencv-python
!pip install tensorboardX

  Using cached https://files.pythonhosted.org/packages/f4/44/60f82c97d1caa98752c7da2c1681cab5c7a390a0fdd3a55fac672b321cac/scikit_optimize-0.5.2-py2.py3-none-any.whl
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
def dict_to_string(hyperdict,logdir="/"):
    s=logdir
    for key,values in hyperdict.items():
        s=s+"_"+key+"_{}_".format(np.round(values, decimals=12))
    return s


In [3]:
wait_epoch=50
max_epoch=500
batch_size=12

writer_name_list_eval=['valid/non_ema_loss_eval','valid/ema_loss_eval','valid/total_loss_eval','valid/lovasz_loss_eval','valid/focal_loss_eval','valid/lovasz_loss_ema_eval',
                       'valid/focal_loss_ema_eval','valid/unsupervised_loss_eval','valid/iou_score_eval']
writer_name_list_train=['train/non_ema_loss','train/ema_loss','train/total_loss','train/lovasz_loss','train/focal_loss','train/lovasz_loss_ema',
                       'train/focal_loss_ema','train/unsupervised_loss','train/iou_score']

#writer = SummaryWriter(log_dir="/home/ubuntu/Kaggle_Pytorch_TGS/plogs2")


In [ ]:
from skopt.space import Real, Integer
from skopt.utils import use_named_args


# The list of hyper-parameters we want to optimize. For each one we define the bounds,
# the corresponding scikit-learn parameter name, as well as how to sample values
# from that dimension (`'log-uniform'` for the learning rate)
space  = [Real(0.1, 0.5, "uniform", name='focal_scaling'),
          Integer(int(1), int(batch_size/2), name='second_batch_size'),
          Real(0.00000001, 0.0001, "log-uniform", name='decay'),
          Real(0.01, 0.3, "uniform", name='unsupervised_scaling'),
          Real(0.1, 0.5, "uniform", name='ema_scaling'),
          Real(0.1, 0.5, "uniform", name='non_ema_scaling'),
          Real(0.01, 0.5, "log-uniform", name='droppout'),
          Real(0.1, 0.5, "uniform", name='lovasz_scaling')]

# this decorator allows your objective function to receive a the parameters as
# keyword arguments. This is particularly convenient when you want to set scikit-learn
# estimator parameters
@use_named_args(space)
def objective(**params):
    logdir = dict_to_string(params)
    logdir="/home/ubuntu/Kaggle_Pytorch_TGS/plogs2"+logdir
    print(logdir)
    writer = SummaryWriter(log_dir=logdir)
    
    augs = iaa.Sequential([
    #iaa.Scale((512, 512)),
    iaa.Fliplr(0.5),
    iaa.Affine(rotate=(-25, 25),mode="reflect",
               translate_percent={"x": (-0.01, 0.01), "y": (-0.01, 0.01)}),
    #iaa.Add((-40, 40), per_channel=0.5, name="color-jitter")  
    ])
    
    
    train_loader,valid_loader=build_loader(input_img_folder='data/train/images/'
                 ,label_folder='data/train/masks/'
                 ,test_img_folder='data/test/images/'
                 ,second_batch_size=params["second_batch_size"]
                 ,batch_size=batch_size
                 ,transform=augs
                 ,show_image=False
                 ,num_workers=4)
    
    
    #in the final training funciotn we will put them too. 
    segmentation_module,segmentation_ema=build_segmentation_model(
    in_arch="resnet50_dilated8",out_arch="upernet" ,droppout=params["droppout"])
    optimizer = torch.optim.SGD(
        group_weight(segmentation_module),
        lr=0.01,
        momentum=0.9,
        weight_decay=params["decay"])
    scheduler = ReduceLROnPlateau(optimizer, 'max')
    
    best_metric=0
    wait=0
    n_iter=0

    
    for j in range(max_epoch):
        #Trains for one epoch
        train(train_loader,segmentation_module,segmentation_ema,optimizer
              ,writer=writer
              ,lovasz_scaling=params["lovasz_scaling"]
              ,focal_scaling=params["focal_scaling"]
              ,unsupervised_scaling=params["unsupervised_scaling"]
              ,ema_scaling=params["ema_scaling"]
              ,non_ema_scaling=params["non_ema_scaling"]
              ,train=True
              #,test=True
              ,writer_name_list=writer_name_list_train
              ,second_batch_size=params["second_batch_size"])
        
        # Does the Evaluation.
        metric=train(valid_loader,segmentation_module,segmentation_ema,optimizer
              ,writer=writer
              ,lovasz_scaling=params["lovasz_scaling"]
              ,focal_scaling=params["focal_scaling"]
              ,unsupervised_scaling=params["unsupervised_scaling"]
              ,ema_scaling=params["ema_scaling"]
              ,non_ema_scaling=params["non_ema_scaling"]
              ,train=False
              #,test=True
              ,writer_name_list=writer_name_list_eval
              ,second_batch_size=params["second_batch_size"])
        scheduler.step(metric)
        
        #Save best metric and do simple early stopping
        if metric > best_metric:
            best_metric=metric
            wait=0
        else:
            wait =wait+1
        if wait > wait_epoch:
            break
    print(best_metric)
    return -best_metric

In [ ]:
res_gp = gp_minimize(objective, space, n_calls=50, random_state=123)


/home/ubuntu/Kaggle_Pytorch_TGS/plogs2/_focal_scaling_0.385182128209__second_batch_size_3__decay_5.801488e-06__unsupervised_scaling_0.218553589945__ema_scaling_0.296447573373__non_ema_scaling_0.412011104765__droppout_0.049905473819__lovasz_scaling_0.331877718809_


/home/ubuntu/Kaggle_Pytorch_TGS/losses.py:163: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  logpt = F.log_softmax(input)
